## Load MNIST on Python 3.x

In [2]:
import pickle
import gzip

In [3]:
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()

## Load USPS on Python 3.x

In [4]:
from PIL import Image
import os
import numpy as np
import keras
from keras.utils import np_utils
from beautifultable import BeautifulTable
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np  
from sklearn.svm import SVC 
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import fetch_mldata 
from sklearn import svm, metrics
from sklearn.model_selection import cross_val_score

C:\Users\RAGHAVA\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

# Logistic Regression with SGD

In [5]:
def softmax(r):
    return np.exp(r)/np.sum(np.exp(r), axis=1).reshape((-1,1))

def loss_cross_entropy(y_predicted, y):
    return -np.sum(y * np.log(y_predicted))

def accuracy(A,Y):
    d = A.argmax(axis=1) - Y.argmax(axis=1)
    a=(1.0 - (float(np.count_nonzero(d)) / len(d)))
    return a*100
  
def logr(TrainingData,TrainingTarget,ValData,ValDataAct,TestData,TestDataAct,TestData_USPS,TestDataAct_USPS):
    table1 = BeautifulTable()
    table1.column_headers = ["Eta","TrA","VaA","TeA","TeA_USPS"] 
#     learningRate = [0.0005,0.001,0.005,0.01,0.1]
    learningRate = [0.0001]
   

    for l in learningRate:
       
        W_Now=np.random.rand(10,784)
    #             print(W_Now.shape)
    #             print(W_Now)
   
        for i in range(0,200):
            for j in range(0,50000):
                    a=softmax(np.dot(TrainingData[j],np.transpose(W_Now)).reshape(-1,10))
#                     loss=loss_cross_entropy(a, TrainingTarget[j:j+100])  
                    D_E= np.dot(np.transpose(TrainingData[j:j+1]),a-TrainingTarget[j])
                    W_Now = W_Now - l*np.transpose(D_E)

#             #-----------------Training Data Accuracy---------------------#
        a_tr=softmax(np.dot(TrainingData,np.transpose(W_Now)))
        TrA=accuracy(a_tr,TrainingTarget)
#         Trloss=loss_cross_entropy(a, TrainingTarget) 

        a_va=softmax(np.dot(ValData,np.transpose(W_Now)))
        VaA=accuracy(a_va,ValDataAct)
#             #-----------------Testing Data Accuracy---------------------#
        a_ts=softmax(np.dot(TestData,np.transpose(W_Now)))
        TeA=accuracy(a_ts,TestDataAct)
       
        a_ts_usps=softmax(np.dot(TestData_USPS,np.transpose(W_Now)))
        TeA_USPS=accuracy(a_ts_usps,TestDataAct_USPS)
           
        table1.append_row([l,float(TrA),float(VaA),float(TeA),float(TeA_USPS)])
    return table1,a_tr,a_va,a_ts,a_ts_usps 
    

In [6]:

table1,c1_tr,c1_va,c1_ts,c1_ts_usps = logr(training_data[0],np_utils.to_categorical(training_data[1], num_classes = 10),validation_data[0],np_utils.to_categorical(validation_data[1], num_classes = 10),test_data[0],np_utils.to_categorical(test_data[1], num_classes = 10),USPSMat,np_utils.to_categorical(USPSTar, num_classes = 10))
print(table1)

+-----+-------+-------+-------+----------+
| Eta |  TrA  |  VaA  |  TeA  | TeA_USPS |
+-----+-------+-------+-------+----------+
| 0.0 | 91.97 | 92.21 | 91.79 |  32.897  |
+-----+-------+-------+-------+----------+


In [7]:
# confusion matrix
print("Logistic SGD Confusion matrix MNIST :n%s" % metrics.confusion_matrix(test_data[1], c1_ts.argmax(axis=1)))
print("Logistic SGD Confusion matrix USPS :n%s" % metrics.confusion_matrix(USPSTar, c1_ts_usps.argmax(axis=1)))


Logistic SGD Confusion matrix MNIST :n[[ 958    0    0    3    0    3    8    3    5    0]
 [   0 1110    2    2    0    3    4    1   13    0]
 [   5   12  907   21    8    4   16   12   41    6]
 [   3    1   17  920    1   25    2    9   23    9]
 [   2    3    7    1  910    0   13    3    9   34]
 [  11    3    7   38   10  753   19    9   34    8]
 [  11    3    4    2   10   12  910    3    3    0]
 [   2    6   23    8    7    1    0  946    3   32]
 [  10    7    6   22    9   34   10   12  861    3]
 [   9    8    1    8   37    7    1   25    9  904]]
Logistic SGD Confusion matrix USPS :n[[ 399    3  251  103  148  367  100   70  116  443]
 [  70  280  217  267  192  154   28  532  237   23]
 [ 105   17 1158  191   39  247   75   51   90   26]
 [  49    3  174 1169   14  441   11   58   58   23]
 [  44   42   46   69  776  151   64  299  301  208]
 [  83    8  203  179   37 1234  101   54   82   19]
 [ 167    6  512   82   69  436  618   15   54   41]
 [ 147  136  149  597  

# Neural Networks

In [6]:
num_classes=10 
image_vector_size=28*28 
I1=[1024]
I2=[2048]
table2 = BeautifulTable()
table2.column_headers = ["1st Layer Hidden Nodes","2nd Layer Hidden Nodes","TrA","VaA","TeA","TeA_USPS"] 

for i in I1: 
    for j in I2:
        y_train= keras.utils.to_categorical(np.concatenate((training_data[1],validation_data[1]), axis=0), num_classes) 
        y_test= keras.utils.to_categorical(test_data[1], num_classes) 
        y_test_usps= keras.utils.to_categorical(USPSTar, num_classes) 
        x_train=np.concatenate((training_data[0],validation_data[0]), axis=0)

        image_size = 784 
        model = Sequential() 
        model.add(Dense(units=i, activation='relu', input_shape=(image_size,))) 
        model.add(Dense(units=j, activation='sigmoid')) 
        model.add(Dense(units=num_classes, activation='softmax')) 
        model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])  
        epochs=100
        history = model.fit(x_train, y_train, batch_size=128, epochs=epochs, verbose=False,validation_split=.1)  
        # %matplotlib inline
#             #-----------------TrainingData Accuracy---------------------#
        df = pd.DataFrame(history.history)
        # df.plot(subplots=True, grid=True, figsize=(10,15))
#         loss,accuracy = model.evaluate(test_data[0], y_test, verbose=False) 
#         loss_usps,accuracy_usps = model.evaluate(np.array(USPSMat),y_test_usps, verbose=False)
        NN_ts=model.predict(test_data[0])
        NN_ts_usps=model.predict(np.array(USPSMat))
#             #-----------------Testing Data Accuracy---------------------#
        accuracy=accuracy_score(test_data[1],NN_ts.argmax(axis=1))
        accuracy_usps=accuracy_score(USPSTar, predicted_usps.argmax(axis=1))
        table2.append_row([i,j,float(df.iloc[epochs-1,3]),float(df.iloc[epochs-1,1]),float(accuracy),float(accuracy_usps)])
print(table2)

+----------------------+----------------------+------+-------+-------+----------+
| 1st Layer Hidden Nod | 2nd Layer Hidden Nod | TrA  |  VaA  |  TeA  | TeA_USPS |
|          es          |          es          |      |       |       |          |
+----------------------+----------------------+------+-------+-------+----------+
|         1024         |         2048         | 0.97 | 0.973 | 0.965 |  0.425   |
+----------------------+----------------------+------+-------+-------+----------+


In [9]:
# confusion matrix
print(" Confusion matrix MNIST :n%s" % metrics.confusion_matrix(test_data[1], NN_ts.argmax(axis=1)))
print(" Confusion matrix USPS :n%s" % metrics.confusion_matrix(USPSTar, NN_ts_usps.argmax(axis=1)))

 Confusion matrix MNIST :n[[ 966    0    1    2    0    4    4    2    1    0]
 [   0 1119    4    0    0    1    4    2    5    0]
 [   5    1 1004    3    3    0    3    7    5    1]
 [   1    0   10  970    0   10    0   10    6    3]
 [   1    1    6    0  944    0    7    2    2   19]
 [   8    1    0    9    2  853    9    1    5    4]
 [   8    3    0    0    7    8  928    1    3    0]
 [   1    9   12    4    2    1    0  988    1   10]
 [   5    2    3    8    4    6    9    8  925    4]
 [   6    6    1    8   17    5    1    7    3  955]]
 Confusion matrix USPS :n[[ 569    1  195   72  145  157   52  103  120  586]
 [  74  379  327  114   87   78   29  760   91   61]
 [  65    8 1485  107   25  144   62   50   41   12]
 [  44    3  142 1427    3  261    5   45   60   10]
 [  23   52   59   33  924  131   21  410  220  127]
 [ 110    4  159  143   20 1365   51   42   92   14]
 [ 142    7  448   56   61  273  907   20   46   40]
 [  66  134  365  444   20  125    6  697  116 



# SVM 

In [7]:
def accuracy_n(A,Y):
    return (1.0 - (float(np.count_nonzero(A-Y)) / len(A-Y)))*100

In [34]:
# SVM  
num_classes=10 
table3 = BeautifulTable()
table3.column_headers = ["Kernel","gamma","TrA","VaA","TeA","TeA_USPS"] 


classifier1 = SVC(kernel='rbf',gamma='auto')
classifier1.fit(training_data[0][0:20000], training_data[1][0:20000])  

# Predict Test Set

Train_Accuracy_SVM=accuracy_score(training_data[1],classifier1.predict(training_data[0]))
Validation_Accuracy_SVM=accuracy_score(validation_data[1],classifier1.predict(validation_data[0]))

predicted = classifier1.predict(test_data[0])
predicted_usps = classifier1.predict(np.array(USPSMat))
Test_Accuracy_SVM=accuracy_score(test_data[1],predicted)
Test_Accuracy_USPS_SVM=accuracy_score(USPSTar, predicted_usps)
table3.append_row(['rbf','default',float(Train_Accuracy_SVM),float(Validation_Accuracy_SVM),float(Test_Accuracy_SVM),float(Test_Accuracy_USPS_SVM)])

print(table3)
# confusion matrix
print("SVM Confusion matrix MNIST :n%s" % metrics.confusion_matrix(test_data[1], predicted))
print("SVM Confusion matrix USPS :n%s" % metrics.confusion_matrix(USPSTar, predicted_usps))



+--------+---------+------+-------+------+----------+
| Kernel |  gamma  | TrA  |  VaA  | TeA  | TeA_USPS |
+--------+---------+------+-------+------+----------+
|  rbf   | default | 0.94 | 0.945 | 0.93 |  0.385   |
+--------+---------+------+-------+------+----------+
SVM Confusion matrix MNIST :n[[ 966    0    1    0    1    5    5    1    1    0]
 [   0 1118    3    2    0    1    4    1    6    0]
 [   9    2  934   10   14    2   18   14   27    2]
 [   2    2   19  932    1   21    1   11   17    4]
 [   1    4    7    0  919    0   10    3    2   36]
 [   7    6    5   36    9  795   14    4   11    5]
 [  10    3    5    1    6   14  918    0    1    0]
 [   3   17   23    4   10    1    0  939    4   27]
 [   4    8    8   18    8   28   11    8  873    8]
 [  10    9    1   12   39    7    1   13    6  911]]
SVM Confusion matrix USPS :n[[ 565    4  409   18  336  223   77   37   11  320]
 [  98  455  240  124  417  190   46  397   19   14]
 [ 107   23 1345   70   48  213   67

# Random Forest

In [10]:
#Trees=[1,3,5,10,15,25,50,75,100,200]
Trees=[200]
table4 = BeautifulTable()
table4.column_headers = ["No. of trees ","TrA","VaA","TeA","TeA_USPS"] 

for i in Trees:
# #RandomForestClassifier  
    classifier2 = RandomForestClassifier(n_estimators=i)
    classifier2.fit(training_data[0], training_data[1])
    
    # Predict Test Set
    Train_Accuracy_RF=accuracy_score(training_data[1],classifier2.predict(training_data[0]))
    Validation_Accuracy_RF=accuracy_score(validation_data[1],classifier2.predict(validation_data[0]))
    predicted1 = classifier2.predict(test_data[0])
    predicted1_usps = classifier2.predict(np.array(USPSMat))
    Test_Accuracy_RF=accuracy_score(test_data[1],predicted1)
    Test_Accuracy_USPS_RF=accuracy_score(USPSTar, predicted1_usps)
    table4.append_row([i,float(Train_Accuracy_RF),float(Validation_Accuracy_RF),float(Test_Accuracy_RF),float(Test_Accuracy_USPS_RF)])
print(table4)
    
# confusion matrix
print("RF Confusion matrix MNIST Random Forest:n%s" % metrics.confusion_matrix(test_data[1], predicted1))
print("RF Confusion matrix USPS Random Forest:n%s" % metrics.confusion_matrix(USPSTar, predicted1_usps))

+----------------------------------------------+-----+-------+------+----------+
|                No. of trees                  | TrA |  VaA  | TeA  | TeA_USPS |
+----------------------------------------------+-----+-------+------+----------+
|                     200                      | 1.0 | 0.974 | 0.97 |  0.401   |
+----------------------------------------------+-----+-------+------+----------+
RF Confusion matrix MNIST Random Forest:n[[ 969    0    0    0    0    2    3    1    4    1]
 [   0 1124    2    3    0    2    2    1    1    0]
 [   6    0  999    7    2    0    4    8    6    0]
 [   0    0    9  972    0    9    0    9    9    2]
 [   1    0    1    0  957    0    4    1    3   15]
 [   2    0    1   15    4  858    5    1    4    2]
 [   7    3    0    0    2    4  938    0    4    0]
 [   1    4   16    1    1    0    0  992    1   12]
 [   3    0    6    8    6    4    5    3  927   12]
 [   7    5    2    8   14    5    1    5    3  959]]
RF Confusion matrix USP

# Mini-batch stochastic gradient descent

In [11]:
def softmax(r):
    return np.exp(r)/np.sum(np.exp(r), axis=1).reshape((-1,1))

def loss_cross_entropy(y_predicted, y):
    return -np.sum(y * np.log(y_predicted))

def accuracy(A,Y):
    d = A.argmax(axis=1) - Y.argmax(axis=1)
    a=(1.0 - (float(np.count_nonzero(d)) / len(d)))
    return a*100
  
def logr_mb_sgd(TrainingData,TrainingTarget,ValData,ValDataAct,TestData,TestDataAct,TestData_USPS,TestDataAct_USPS):
    table5 = BeautifulTable()
    table5.column_headers = ["Eta","TrA","VaA","TeA","TeA_USPS"] 
    learningRate = [0.0001]
#     learningRate = [0.01]
   

    for l in learningRate:
       
        W_Now=np.random.rand(10,784)
    #             print(W_Now.shape)
    #             print(W_Now)
   
        for i in range(0,200):
            batch_size=64
            for j in range(0,50000,batch_size):
                    a=softmax(np.dot(TrainingData[j:j+batch_size],np.transpose(W_Now)))
#                     loss=loss_cross_entropy(a, TrainingTarget[j:j+100])  
                    D_E= np.dot(np.transpose(TrainingData[j:j+batch_size]),a-TrainingTarget[j:j+batch_size])
                    W_Now = W_Now - l*np.transpose(D_E)

#             #-----------------TrainingData Accuracy---------------------#
        a5_tr=softmax(np.dot(TrainingData,np.transpose(W_Now)))
        TrA=accuracy(a5_tr,TrainingTarget)
#         Trloss=loss_cross_entropy(a, TrainingTarget) 

        a5_va=softmax(np.dot(ValData,np.transpose(W_Now)))
        VaA=accuracy(a5_va,ValDataAct)
#             #-----------------Testing Data Accuracy---------------------#

        a5_ts=softmax(np.dot(TestData,np.transpose(W_Now)))
        TeA=accuracy(a5_ts,TestDataAct)
       
        a5_ts_usps=softmax(np.dot(TestData_USPS,np.transpose(W_Now)))
        TeA_USPS=accuracy(a5_ts_usps,TestDataAct_USPS)
           
        table5.append_row([l,float(TrA),float(VaA),float(TeA),float(TeA_USPS)])
    return table5,a5_tr,a5_va,a5_ts,a5_ts_usps
    

In [12]:

table5,c5_tr,c5_va,c5_ts,c5_ts_usps= logr_mb_sgd(training_data[0],np_utils.to_categorical(training_data[1], num_classes = 10),validation_data[0],np_utils.to_categorical(validation_data[1], num_classes = 10),test_data[0],np_utils.to_categorical(test_data[1], num_classes = 10),USPSMat,np_utils.to_categorical(USPSTar, num_classes = 10))
print(table5)
# confusion matrix
print("Logistic SGD Confusion matrix MNIST :n%s" % metrics.confusion_matrix(test_data[1], c5_ts.argmax(axis=1)))
print("Logistic SGD Confusion matrix USPS :n%s" % metrics.confusion_matrix(USPSTar, c5_ts_usps.argmax(axis=1)))


+-----+--------+-------+-------+----------+
| Eta |  TrA   |  VaA  |  TeA  | TeA_USPS |
+-----+--------+-------+-------+----------+
| 0.0 | 91.952 | 92.37 | 91.94 |  32.482  |
+-----+--------+-------+-------+----------+
Logistic SGD Confusion matrix MNIST :n[[ 956    0    1    2    0    7   11    1    2    0]
 [   0 1111    2    2    0    2    4    1   13    0]
 [   8    9  914   18    9    1   12   11   41    9]
 [   1    0   18  916    0   29    2   12   25    7]
 [   2    3    4    2  919    0    7    2    7   36]
 [  10    3    5   43   10  756   16    6   37    6]
 [   8    3    5    2   10   17  909    2    2    0]
 [   2    5   20   10   10    0    0  947    6   28]
 [   7    7    9   23    9   31    9   14  861    4]
 [  10    6    1   10   38    7    0   27    5  905]]
Logistic SGD Confusion matrix USPS :n[[ 478    2  324   94  119  293   94  156  205  235]
 [  85  266  289  191  208  105   20  594  222   20]
 [ 131   11 1266  128   30  177   75   63   94   24]
 [  69    3  24

# Ensemble Methods - Majority/Hard Voting


In [31]:
Classifier1_ts=c1_ts
Classifier1_ts_usps= c1_ts_usps
Classifier2_ts=NN_ts
Classifier2_ts_usps=NN_ts_usps
Classifier3_ts=np_utils.to_categorical(predicted, num_classes = 10)
Classifier3_ts_usps=np_utils.to_categorical(predicted_usps, num_classes = 10)
Classifier4_ts=np_utils.to_categorical(predicted1, num_classes = 10)
Classifier4_ts_usps=np_utils.to_categorical(predicted1_usps, num_classes = 10)
Classifier5_ts=c5_ts
Classifier5_ts_usps=c5_ts_usps

#Logic for Majority/ Hard Voting
Classifier_test=Classifier1_ts+Classifier2_ts+Classifier3_ts+Classifier4_ts+Classifier5_ts
Classifier_test_usps=Classifier1_ts_usps+Classifier2_ts_usps+Classifier3_ts_usps+Classifier4_ts_usps+Classifier5_ts_usps


Majority_Voting_test = accuracy_score(test_data[1],Classifier_test.argmax(axis=1))
Majority_Voting_test_usps= accuracy_score(USPSTar,Classifier_test_usps.argmax(axis=1))
print("Majority Voting MNIST Test accuracy is " + str(Majority_Voting_test))
print("Majority Voting USPS Test accuracy is " +str(Majority_Voting_test_usps))

# confusion matrix
print("Majority_Voting Confusion matrix MNIST :n%s" % metrics.confusion_matrix(test_data[1], Classifier_test.argmax(axis=1)))
print("Majority_Voting Confusion matrix USPS :n%s" % metrics.confusion_matrix(USPSTar, Classifier_test_usps.argmax(axis=1)))



Majority Voting MNIST Test accuracy is 0.9675
Majority Voting USPS Test accuracy is 0.4224211210560528
Majority_Voting Confusion matrix MNIST :n[[ 968    0    1    1    0    3    4    1    2    0]
 [   0 1123    2    2    0    1    3    1    3    0]
 [   7    1  997    5    3    0    3    8    7    1]
 [   0    0    9  973    0    7    0   11    8    2]
 [   1    0    3    0  955    0    6    2    2   13]
 [   7    1    0   12    2  854    7    1    6    2]
 [   7    3    0    0    4    8  933    1    2    0]
 [   1    7   15    2    2    1    0  989    0   11]
 [   4    1    4    9    5    5    8    8  926    4]
 [   8    6    1    9   14    3    1    7    3  957]]
Majority_Voting Confusion matrix USPS :n[[ 586    2  245   69  209  191   57  103   78  460]
 [  65  413  270  114   56   87   24  851   83   37]
 [  77   10 1478   93   29  149   43   86   28    6]
 [  38    3  131 1411    4  290    4   63   48    8]
 [  17   77   56   22  991  150   17  367  202  101]
 [  99   11  181  12